````
LeaderboardCyclopeptideSequencing(Spectrum, N)
    Leaderboard ← set containing only the empty peptide
    LeaderPeptide ← empty peptide
    while Leaderboard is non-empty
        Leaderboard ← Expand(Leaderboard)
        for each Peptide in Leaderboard
            if Mass(Peptide) = ParentMass(Spectrum)
                if Score(Peptide, Spectrum) > Score(LeaderPeptide, Spectrum)
                    LeaderPeptide ← Peptide
            else if Mass(Peptide) > ParentMass(Spectrum)
                remove Peptide from Leaderboard
        Leaderboard ← Trim(Leaderboard, Spectrum, N)
    output LeaderPeptide
````

Desafio de código: implementar o LeaderboardCyclopeptideSequencing.

* Entrada: Um inteiro N e uma coleção de inteiros Spectrum .
* Saída: LeaderPeptide após executar 'LeaderboardCyclopeptideSequencing(Spectrum,N)'.

#### LeaderboardCyclopeptideSequencing(Spectrum, N):

In [5]:
def Expand_2(Peptides):
    expanded = []
    for peptide in Peptides:
        for aa in AminoAcidMass.keys():
            expanded.append(peptide + aa)
    return expanded

def ParentMass_2(Spectrum):
    return max(Spectrum)  # A maior massa no espectro

def Mass_2(Peptide):
    return sum(AminoAcidMass[aa] for aa in Peptide)  # Massa total de um peptídeo

def CyclicSpectrum_2(Peptide, Alphabet, AminoAcidMass): # Comentários do código em Bioinfo_II_3
    PrefixMass = [0] * (len(Peptide) + 1)

    for i in range(1, len(Peptide) + 1):
        for s in Alphabet:
            if s == Peptide[i - 1]:
                PrefixMass[i] = PrefixMass[i - 1] + AminoAcidMass[s]

    peptideMass = PrefixMass[len(Peptide)]
    CyclicSpectrum = [0]

    for i in range(len(Peptide)):
        for j in range(i + 1, len(Peptide) + 1):
            CyclicSpectrum.append(PrefixMass[j] - PrefixMass[i])
            if i > 0 and j < len(Peptide):
                CyclicSpectrum.append(
                    peptideMass - (PrefixMass[j] - PrefixMass[i]))

    return sorted(CyclicSpectrum)

def CyclicScore_2(Peptide, Spectrum, Alphabet, AminoAcidMass):
    Spectrum_Teor = CyclicSpectrum_2(Peptide, Alphabet, AminoAcidMass)
    Score_final_Cyclic = 0
    p1 = 0
    p2 = 0
    
    while p1 < len(Spectrum_Teor) and p2 < len(Spectrum):
        if Spectrum_Teor[p1] == Spectrum[p2]:
            Score_final_Cyclic += 1
            p1 += 1
            p2 += 1
        elif Spectrum_Teor[p1] < Spectrum[p2]:
            p1 += 1
        elif Spectrum_Teor[p1] > Spectrum[p2]:
            p2 +=1
    return Score_final_Cyclic


def LinearSpectrum_2(Peptide, Alphabet, AminoAcidMass):
    PrefixMass = [0] * (len(Peptide) + 1)
    for i in range(1, len(Peptide) + 1):
        for s in Alphabet:
            if s == Peptide[i - 1]: 
                PrefixMass[i] = PrefixMass[i - 1] + AminoAcidMass[s]
    LinearSpectrum = [0]

    for i in range(len(Peptide)):
        for j in range(i + 1, len(Peptide) + 1):
            LinearSpectrum.append(PrefixMass[j] - PrefixMass[i])

    return sorted(LinearSpectrum)

def LinearScore_2(Peptide, Spectrum, Alphabet, AminoAcidMass):
    Spectrum_Teor = LinearSpectrum_2(Peptide, Alphabet, AminoAcidMass)
    LinearScore_final_linear = 0
    p1 = 0
    p2 = 0
    
    while p1 < len(Spectrum_Teor) and p2 < len(Spectrum):
        if Spectrum_Teor[p1] == Spectrum[p2]:
            LinearScore_final_linear += 1
            p1 += 1
            p2 += 1
        elif Spectrum_Teor[p1] < Spectrum[p2]:
            p1 += 1
        elif Spectrum_Teor[p1] > Spectrum[p2]:
            p2 +=1
    return LinearScore_final_linear

def Trim_2(Leaderboard, Spectrum, N, Alphabet, AminoAcidMass):
    # Passo 1: Calcular LinearScores
    LinearScores = []
    for j in range(1, len(Leaderboard) + 1):  # Loop começa de 1 para seguir o pseudocódigo
        Peptide = Leaderboard[j - 1]  # j-ésimo peptídeo (índice começa em 0 no Python)
        LinearScores.append(LinearScore_2(Peptide, Spectrum, Alphabet, AminoAcidMass))

    sorted_indices = sorted(range(len(LinearScores)), key=lambda x: -LinearScores[x])  # Índices ordenados
    Leaderboard = [Leaderboard[i] for i in sorted_indices]  # Reordena Leaderboard
    LinearScores = [LinearScores[i] for i in sorted_indices]  # Reordena LinearScores

    for j in range(N + 1, len(Leaderboard) + 1):  # j vai de N+1 até |Leaderboard| (inclusive)
        if LinearScores[j - 1] < LinearScores[N - 1]:  # Índices ajustados para Python
            return Leaderboard[:j - 1]  # Trunca Leaderboard até o índice anterior

    return Leaderboard

def LeaderboardCyclopeptideSequencing_2(Spectrum, N):
    Leaderboard = {""}  # Conjunto contendo apenas o peptídeo vazio
    LeaderPeptide = ""  # Peptídeo líder inicialmente vazio

    while Leaderboard:
        Leaderboard = Expand_2(Leaderboard)  # Expande os peptídeos no Leaderboard
        for Peptide in list(Leaderboard):  # Itera sobre os peptídeos
            if Mass_2(Peptide) == ParentMass_2(Spectrum):
                if CyclicScore_2(Peptide, Spectrum,Alphabet, AminoAcidMass) > CyclicScore_2(LeaderPeptide, Spectrum, Alphabet, AminoAcidMass):
                    LeaderPeptide = Peptide
            elif Mass_2(Peptide) > ParentMass_2(Spectrum):
                Leaderboard.remove(Peptide)  # Remove peptídeos com massa maior
        Leaderboard = Trim_2(Leaderboard, Spectrum, N, Alphabet, AminoAcidMass)  # Aplica o Trim

    return LeaderPeptide

def format_peptide_2(peptide):
    masses = [str(AminoAcidMass[aa]) for aa in peptide]
    return "-".join(masses)

with open("dataset_30244_10.txt", "r") as file:
    lines = file.readlines()
    N = int(lines[0].strip())  # Primeiro valor é N
    Spectrum = list(map(int, lines[1].strip().split()))  # Segunda linha é o Spectrum


Alphabet = ['G', 'A', 'S', 'P', 'V', 'T', 'C', 'I', 'L', 'N', 'D', 'K', 'Q', 'E', 'M', 'H', 'F', 'R', 'Y', 'W'] 
AminoAcidMass = {'G': 57, 'A': 71, 'S': 87, 'P': 97,
                 'V': 99, 'T': 101, 'C': 103, 'I': 113,
                 'L': 113, 'N': 114, 'D': 115, 'K': 128,
                 'Q': 128, 'E': 129, 'M': 131, 'H': 137,
                 'F': 147, 'R': 156, 'Y': 163, 'W': 186}

result = LeaderboardCyclopeptideSequencing_2(Spectrum, N)
print(result)
print(format_peptide_2(result))

IKVYKNFDAPF
113-128-99-163-128-114-147-115-71-97-147


: 

#### Código que funcionou para o exercício:
  Retorne seus peptídeos em formato inteiro, com cada peptídeo separado por um único espaço, por exemplo, 113-147-71-129 71-147-129-113.

````
LeaderboardCyclopeptideSequencing(Spectrum, N)
    Leaderboard ← set containing only the empty peptide
    LeaderPeptides ← empty list
    MaxScore ← 0

    while Leaderboard is not empty
        Leaderboard ← Expand(Leaderboard)
        for each Peptide in Leaderboard
            if Mass(Peptide) = ParentMass(Spectrum)
                CyclicScore ← CyclicScore(Peptide, Spectrum, Alphabet, AminoAcidMass)
                if CyclicScore > MaxScore
                    MaxScore ← CyclicScore
                    LeaderPeptides ← list containing only Peptide
                else if CyclicScore = MaxScore
                    append Peptide to LeaderPeptides
            else if Mass(Peptide) > ParentMass(Spectrum)
                remove Peptide from Leaderboard
        Leaderboard ← Trim(Leaderboard, Spectrum, N, Alphabet, AminoAcidMass)
    return sorted(LeaderPeptides)

````

In [4]:
def LeaderboardCyclopeptideSequencing(Spectrum, N):
    Leaderboard = {""}  # Conjunto contendo apenas o peptídeo vazio
    LeaderPeptides = []  # Lista para armazenar os peptídeos líderes de máximo score
    MaxScore = 0  # Maior score encontrado // MODIFICAÇÃO PARA ESSE EXERCÍCIO
    
    while Leaderboard:
        Leaderboard = Expand(Leaderboard)  # Expande os peptídeos no Leaderboard
        for Peptide in list(Leaderboard):  # Itera sobre os peptídeos
            if Mass(Peptide) == ParentMass(Spectrum):
                score = CyclicScore(Peptide, Spectrum, Alphabet, AminoAcidMass)
                if score > MaxScore:
                    MaxScore = score
                    LeaderPeptides = [Peptide]  # Atualiza os peptídeos líderes
                elif score == MaxScore:
                    LeaderPeptides.append(Peptide)
            elif Mass(Peptide) > ParentMass(Spectrum):
                Leaderboard.remove(Peptide)  # Remove peptídeos com massa maior
        Leaderboard = Trim(Leaderboard, Spectrum, N, Alphabet, AminoAcidMass)

    return sorted(LeaderPeptides)  # Garantir saída ordenada

def format_peptide(peptide):
    masses = [str(AminoAcidMass[aa]) for aa in peptide]
    return "-".join(masses)

def Mass(Peptide):
    return sum(AminoAcidMass[aa] for aa in Peptide)

def ParentMass(Spectrum):
    return max(Spectrum)

def Expand(Peptides):
    expanded = []
    for peptide in Peptides:
        for aa in AminoAcidMass.keys():
            expanded.append(peptide + aa)
    return expanded

def CyclicSpectrum(Peptide, Alphabet, AminoAcidMass):
    PrefixMass = [0] * (len(Peptide) + 1)

    for i in range(1, len(Peptide) + 1):
        for s in Alphabet:
            if s == Peptide[i - 1]:
                PrefixMass[i] = PrefixMass[i - 1] + AminoAcidMass[s]

    peptideMass = PrefixMass[len(Peptide)]
    CyclicSpectrum = [0]

    for i in range(len(Peptide)):
        for j in range(i + 1, len(Peptide) + 1):
            CyclicSpectrum.append(PrefixMass[j] - PrefixMass[i])
            if i > 0 and j < len(Peptide):
                CyclicSpectrum.append(
                    peptideMass - (PrefixMass[j] - PrefixMass[i]))

    return sorted(CyclicSpectrum)

def CyclicScore(Peptide, Spectrum, Alphabet, AminoAcidMass):
    Spectrum_Teor = CyclicSpectrum(Peptide, Alphabet, AminoAcidMass)
    score = 0
    i, j = 0, 0
    while i < len(Spectrum_Teor) and j < len(Spectrum):
        if Spectrum_Teor[i] == Spectrum[j]:
            score += 1
            i += 1
            j += 1
        elif Spectrum_Teor[i] < Spectrum[j]:
            i += 1
        else:
            j += 1
    return score

def Trim(Leaderboard, Spectrum, N, Alphabet, AminoAcidMass):
    LinearScores = []
    for peptide in Leaderboard:
        LinearScores.append(LinearScore(peptide, Spectrum, Alphabet, AminoAcidMass))
    sorted_indices = sorted(range(len(LinearScores)), key=lambda x: -LinearScores[x])
    Leaderboard = [Leaderboard[i] for i in sorted_indices]
    LinearScores = [LinearScores[i] for i in sorted_indices]
    if len(Leaderboard) > N:
        cutoff = LinearScores[N - 1]
        Leaderboard = [Leaderboard[i] for i in range(len(Leaderboard)) if LinearScores[i] >= cutoff]
    return Leaderboard

def LinearSpectrum(Peptide, Alphabet, AminoAcidMass):
    PrefixMass = [0] * (len(Peptide) + 1)
    for i in range(1, len(Peptide) + 1):
        for s in Alphabet:
            if s == Peptide[i - 1]: 
                PrefixMass[i] = PrefixMass[i - 1] + AminoAcidMass[s]
    LinearSpectrum = [0]

    for i in range(len(Peptide)):
        for j in range(i + 1, len(Peptide) + 1):
            LinearSpectrum.append(PrefixMass[j] - PrefixMass[i])

    return sorted(LinearSpectrum)

def LinearScore(Peptide, Spectrum, Alphabet, AminoAcidMass):
    Spectrum_Teor = LinearSpectrum(Peptide, Alphabet, AminoAcidMass)
    score = 0
    i, j = 0, 0
    while i < len(Spectrum_Teor) and j < len(Spectrum):
        if Spectrum_Teor[i] == Spectrum[j]:
            score += 1
            i += 1
            j += 1
        elif Spectrum_Teor[i] < Spectrum[j]:
            i += 1
        else:
            j += 1
    return score

with open("dataset_30244_10.txt", "r") as file:
    lines = file.readlines()
    N = int(lines[0].strip()) 
    Spectrum = list(map(int, lines[1].strip().split())) 

result_peptides = LeaderboardCyclopeptideSequencing(Spectrum, N)
formatted_result = " ".join(format_peptide(peptide) for peptide in result_peptides)
print(formatted_result)

71-115-147-114-128-163-99-128-113-147-97 71-115-147-114-128-163-99-128-113-147-97 71-115-147-114-128-163-99-128-113-147-97 71-115-147-114-128-163-99-128-113-147-97 71-115-147-114-128-163-99-128-113-147-97 71-115-147-114-128-163-99-128-113-147-97 71-115-147-114-128-163-99-128-113-147-97 71-115-147-114-128-163-99-128-113-147-97 115-71-147-114-128-163-99-128-113-147-97 115-71-147-114-128-163-99-128-113-147-97 115-71-147-114-128-163-99-128-113-147-97 115-71-147-114-128-163-99-128-113-147-97 115-71-147-114-128-163-99-128-113-147-97 115-71-147-114-128-163-99-128-113-147-97 115-71-147-114-128-163-99-128-113-147-97 115-71-147-114-128-163-99-128-113-147-97 147-113-128-99-163-128-114-147-115-71-97 147-113-128-99-163-128-114-147-115-71-97 147-113-128-99-163-128-114-147-115-71-97 147-113-128-99-163-128-114-147-115-71-97 147-113-128-99-163-128-114-147-115-71-97 147-113-128-99-163-128-114-147-115-71-97 147-113-128-99-163-128-114-147-115-71-97 147-113-128-99-163-128-114-147-115-71-97 147-114-128-163-